# Capstone Project: Task Manager Django Web Application

Build a full-featured Task Manager web application using Django, demonstrating core web development concepts including models, views, templates, forms, and URL routing.

## Project Overview

### What We're Building
A **Task Manager** application that allows users to:
- Create, read, update, and delete tasks (CRUD operations)
- Organize tasks into categories
- Set priorities and due dates
- Track task status (pending, in progress, completed)

### Skills Applied
- **Django Framework**: Project structure, settings, apps
- **Models & ORM**: Database design, relationships, migrations
- **Class-Based Views (CBVs)**: ListView, DetailView, CreateView, UpdateView, DeleteView
- **URL Routing**: Named URLs, URL patterns
- **Templates**: Template inheritance, context, template tags
- **Forms**: ModelForms, form validation
- **Admin Interface**: Model registration, customization

### Prerequisites
- Python 3.8+
- Django 4.0+ (`pip install django`)
- Basic understanding of Python, HTML, and web concepts
- A code editor (VS Code, PyCharm, etc.)

### Project Structure
```
task_manager_project/
    manage.py
    task_manager/
        __init__.py
        settings.py
        urls.py
        wsgi.py
        asgi.py
    tasks/
        __init__.py
        admin.py
        apps.py
        forms.py
        models.py
        urls.py
        views.py
        templates/
            tasks/
                base.html
                task_list.html
                task_detail.html
                task_form.html
                task_confirm_delete.html
        migrations/
```

---
## Section 1: Project Setup

### Step 1.1: Create the Django Project

Open your terminal and run the following commands:

In [ ]:
# Terminal commands to set up the project
# Run these in your terminal (not in Jupyter)

# 1. Create a directory for the project
# mkdir task_manager_project
# cd task_manager_project

# 2. Create a virtual environment (recommended)
# python -m venv venv
# On Windows: venv\Scripts\activate
# On macOS/Linux: source venv/bin/activate

# 3. Install Django
# pip install django

# 4. Create the Django project
# django-admin startproject task_manager .

# 5. Create the tasks app
# python manage.py startapp tasks

print("Project setup commands listed above - run in terminal")

### Step 1.2: Configure Settings

Update `task_manager/settings.py` to register the tasks app:

In [ ]:
# task_manager/settings.py (relevant sections)

SETTINGS_CODE = '''
# Add 'tasks' to INSTALLED_APPS
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'tasks',  # Add this line
]

# Database configuration (SQLite by default, fine for development)
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

# Time zone configuration
TIME_ZONE = 'UTC'  # Change to your timezone, e.g., 'America/New_York'
USE_TZ = True

# Static files
STATIC_URL = 'static/'
'''

print(SETTINGS_CODE)

---
## Section 2: Models

Create the database models for Category and Task.

### Step 2.1: Define Models

Create `tasks/models.py`:

In [ ]:
# tasks/models.py

MODELS_CODE = '''
from django.db import models
from django.urls import reverse
from django.utils import timezone


class Category(models.Model):
    """Category for organizing tasks.
    
    Attributes:
        name: The category name (unique).
        description: Optional description of the category.
        created_at: Timestamp when category was created.
    """
    name = models.CharField(max_length=100, unique=True)
    description = models.TextField(blank=True)
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        verbose_name_plural = "categories"
        ordering = ['name']
    
    def __str__(self) -> str:
        return self.name


class Task(models.Model):
    """A task item in the task manager.
    
    Attributes:
        title: The task title.
        description: Detailed description of the task.
        status: Current status (pending, in_progress, completed).
        priority: Task priority (low, medium, high).
        due_date: Optional deadline for the task.
        category: Optional category for organization.
        created_at: Timestamp when task was created.
        updated_at: Timestamp of last update.
    """
    
    # Status choices
    STATUS_PENDING = 'pending'
    STATUS_IN_PROGRESS = 'in_progress'
    STATUS_COMPLETED = 'completed'
    
    STATUS_CHOICES = [
        (STATUS_PENDING, 'Pending'),
        (STATUS_IN_PROGRESS, 'In Progress'),
        (STATUS_COMPLETED, 'Completed'),
    ]
    
    # Priority choices
    PRIORITY_LOW = 'low'
    PRIORITY_MEDIUM = 'medium'
    PRIORITY_HIGH = 'high'
    
    PRIORITY_CHOICES = [
        (PRIORITY_LOW, 'Low'),
        (PRIORITY_MEDIUM, 'Medium'),
        (PRIORITY_HIGH, 'High'),
    ]
    
    # Fields
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    status = models.CharField(
        max_length=20,
        choices=STATUS_CHOICES,
        default=STATUS_PENDING
    )
    priority = models.CharField(
        max_length=10,
        choices=PRIORITY_CHOICES,
        default=PRIORITY_MEDIUM
    )
    due_date = models.DateField(null=True, blank=True)
    category = models.ForeignKey(
        Category,
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name='tasks'
    )
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        ordering = ['-created_at']
    
    def __str__(self) -> str:
        return self.title
    
    def get_absolute_url(self) -> str:
        """Return the URL for this task's detail view."""
        return reverse('tasks:task_detail', kwargs={'pk': self.pk})
    
    @property
    def is_overdue(self) -> bool:
        """Check if the task is past its due date."""
        if self.due_date and self.status != self.STATUS_COMPLETED:
            return self.due_date < timezone.now().date()
        return False
'''

print(MODELS_CODE)

### Step 2.2: Create and Apply Migrations

Run these commands in your terminal:

In [ ]:
# Terminal commands for migrations

# Create migrations
# python manage.py makemigrations tasks

# Apply migrations
# python manage.py migrate

print("Migration commands:")
print("1. python manage.py makemigrations tasks")
print("2. python manage.py migrate")

---
## Section 3: Admin Interface

Register models with the Django admin for easy data management.

### Step 3.1: Configure Admin

Update `tasks/admin.py`:

In [ ]:
# tasks/admin.py

ADMIN_CODE = '''
from django.contrib import admin
from .models import Category, Task


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    """Admin configuration for Category model."""
    list_display = ['name', 'description', 'created_at']
    search_fields = ['name', 'description']
    list_filter = ['created_at']


@admin.register(Task)
class TaskAdmin(admin.ModelAdmin):
    """Admin configuration for Task model."""
    list_display = ['title', 'status', 'priority', 'category', 'due_date', 'created_at']
    list_filter = ['status', 'priority', 'category', 'created_at']
    search_fields = ['title', 'description']
    date_hierarchy = 'created_at'
    list_editable = ['status', 'priority']
    
    fieldsets = (
        (None, {
            'fields': ('title', 'description')
        }),
        ('Status & Priority', {
            'fields': ('status', 'priority', 'due_date')
        }),
        ('Organization', {
            'fields': ('category',)
        }),
    )
'''

print(ADMIN_CODE)

### Step 3.2: Create Superuser

Create an admin user to access the admin interface:

In [ ]:
# Terminal command to create superuser
# python manage.py createsuperuser

# Follow the prompts to enter:
# - Username
# - Email (optional)
# - Password

print("Create superuser: python manage.py createsuperuser")
print("Then access admin at: http://127.0.0.1:8000/admin/")

---
## Section 4: Forms

Create a ModelForm for task creation and editing.

### Step 4.1: Create TaskForm

Create `tasks/forms.py`:

In [ ]:
# tasks/forms.py

FORMS_CODE = '''
from django import forms
from .models import Task, Category


class TaskForm(forms.ModelForm):
    """Form for creating and updating Task objects.
    
    Provides form fields with appropriate widgets and validation
    for the Task model.
    """
    
    class Meta:
        model = Task
        fields = ['title', 'description', 'status', 'priority', 'due_date', 'category']
        widgets = {
            'title': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Enter task title'
            }),
            'description': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 4,
                'placeholder': 'Enter task description (optional)'
            }),
            'status': forms.Select(attrs={
                'class': 'form-control'
            }),
            'priority': forms.Select(attrs={
                'class': 'form-control'
            }),
            'due_date': forms.DateInput(attrs={
                'class': 'form-control',
                'type': 'date'
            }),
            'category': forms.Select(attrs={
                'class': 'form-control'
            }),
        }
    
    def clean_title(self) -> str:
        """Validate that title is not empty or whitespace only."""
        title = self.cleaned_data.get('title', '').strip()
        if not title:
            raise forms.ValidationError('Title cannot be empty or whitespace only.')
        return title


class CategoryForm(forms.ModelForm):
    """Form for creating and updating Category objects."""
    
    class Meta:
        model = Category
        fields = ['name', 'description']
        widgets = {
            'name': forms.TextInput(attrs={
                'class': 'form-control',
                'placeholder': 'Enter category name'
            }),
            'description': forms.Textarea(attrs={
                'class': 'form-control',
                'rows': 3,
                'placeholder': 'Enter category description (optional)'
            }),
        }
'''

print(FORMS_CODE)

---
## Section 5: Views

Create Class-Based Views (CBVs) for all CRUD operations.

### Step 5.1: Create Views

Update `tasks/views.py`:

In [ ]:
# tasks/views.py

VIEWS_CODE = '''
from django.views.generic import (
    ListView,
    DetailView,
    CreateView,
    UpdateView,
    DeleteView
)
from django.urls import reverse_lazy
from django.contrib import messages
from .models import Task, Category
from .forms import TaskForm, CategoryForm


class TaskListView(ListView):
    """Display a list of all tasks.
    
    Provides filtering by status and category via query parameters.
    Template: tasks/task_list.html
    """
    model = Task
    template_name = 'tasks/task_list.html'
    context_object_name = 'tasks'
    paginate_by = 10
    
    def get_queryset(self):
        """Filter tasks based on query parameters."""
        queryset = super().get_queryset()
        
        # Filter by status
        status = self.request.GET.get('status')
        if status:
            queryset = queryset.filter(status=status)
        
        # Filter by priority
        priority = self.request.GET.get('priority')
        if priority:
            queryset = queryset.filter(priority=priority)
        
        # Filter by category
        category = self.request.GET.get('category')
        if category:
            queryset = queryset.filter(category_id=category)
        
        return queryset
    
    def get_context_data(self, **kwargs):
        """Add filter options to context."""
        context = super().get_context_data(**kwargs)
        context['categories'] = Category.objects.all()
        context['status_choices'] = Task.STATUS_CHOICES
        context['priority_choices'] = Task.PRIORITY_CHOICES
        context['current_status'] = self.request.GET.get('status', '')
        context['current_priority'] = self.request.GET.get('priority', '')
        context['current_category'] = self.request.GET.get('category', '')
        return context


class TaskDetailView(DetailView):
    """Display details of a single task.
    
    Template: tasks/task_detail.html
    """
    model = Task
    template_name = 'tasks/task_detail.html'
    context_object_name = 'task'


class TaskCreateView(CreateView):
    """Create a new task.
    
    Uses TaskForm for validation.
    Redirects to task detail on success.
    Template: tasks/task_form.html
    """
    model = Task
    form_class = TaskForm
    template_name = 'tasks/task_form.html'
    
    def form_valid(self, form):
        """Add success message on task creation."""
        messages.success(self.request, 'Task created successfully!')
        return super().form_valid(form)
    
    def get_context_data(self, **kwargs):
        """Add page title to context."""
        context = super().get_context_data(**kwargs)
        context['page_title'] = 'Create New Task'
        context['button_text'] = 'Create Task'
        return context


class TaskUpdateView(UpdateView):
    """Update an existing task.
    
    Uses TaskForm for validation.
    Redirects to task detail on success.
    Template: tasks/task_form.html
    """
    model = Task
    form_class = TaskForm
    template_name = 'tasks/task_form.html'
    
    def form_valid(self, form):
        """Add success message on task update."""
        messages.success(self.request, 'Task updated successfully!')
        return super().form_valid(form)
    
    def get_context_data(self, **kwargs):
        """Add page title to context."""
        context = super().get_context_data(**kwargs)
        context['page_title'] = f'Edit Task: {self.object.title}'
        context['button_text'] = 'Update Task'
        return context


class TaskDeleteView(DeleteView):
    """Delete a task with confirmation.
    
    Redirects to task list on success.
    Template: tasks/task_confirm_delete.html
    """
    model = Task
    template_name = 'tasks/task_confirm_delete.html'
    success_url = reverse_lazy('tasks:task_list')
    context_object_name = 'task'
    
    def form_valid(self, form):
        """Add success message on task deletion."""
        messages.success(self.request, 'Task deleted successfully!')
        return super().form_valid(form)


# Category Views (bonus)
class CategoryListView(ListView):
    """Display a list of all categories."""
    model = Category
    template_name = 'tasks/category_list.html'
    context_object_name = 'categories'


class CategoryCreateView(CreateView):
    """Create a new category."""
    model = Category
    form_class = CategoryForm
    template_name = 'tasks/category_form.html'
    success_url = reverse_lazy('tasks:category_list')
    
    def form_valid(self, form):
        messages.success(self.request, 'Category created successfully!')
        return super().form_valid(form)
'''

print(VIEWS_CODE)

---
## Section 6: URL Configuration

Set up URL routing for the application.

### Step 6.1: App URLs

Create `tasks/urls.py`:

In [ ]:
# tasks/urls.py

APP_URLS_CODE = '''
from django.urls import path
from . import views

app_name = 'tasks'

urlpatterns = [
    # Task URLs
    path('', views.TaskListView.as_view(), name='task_list'),
    path('task/<int:pk>/', views.TaskDetailView.as_view(), name='task_detail'),
    path('task/create/', views.TaskCreateView.as_view(), name='task_create'),
    path('task/<int:pk>/update/', views.TaskUpdateView.as_view(), name='task_update'),
    path('task/<int:pk>/delete/', views.TaskDeleteView.as_view(), name='task_delete'),
    
    # Category URLs
    path('categories/', views.CategoryListView.as_view(), name='category_list'),
    path('category/create/', views.CategoryCreateView.as_view(), name='category_create'),
]
'''

print(APP_URLS_CODE)

### Step 6.2: Project URLs

Update `task_manager/urls.py`:

In [ ]:
# task_manager/urls.py

PROJECT_URLS_CODE = '''
from django.contrib import admin
from django.urls import path, include
from django.views.generic import RedirectView

urlpatterns = [
    path('admin/', admin.site.urls),
    path('tasks/', include('tasks.urls')),
    path('', RedirectView.as_view(url='/tasks/', permanent=False)),  # Redirect root to tasks
]
'''

print(PROJECT_URLS_CODE)

---
## Section 7: Templates

Create the HTML templates for the application.

### Step 7.1: Create Template Directory

Create the directory structure:
```
tasks/
    templates/
        tasks/
            base.html
            task_list.html
            task_detail.html
            task_form.html
            task_confirm_delete.html
```

### Step 7.2: Base Template

Create `tasks/templates/tasks/base.html`:

In [ ]:
# tasks/templates/tasks/base.html

BASE_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Task Manager{% endblock %}</title>
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .priority-high { color: #dc3545; font-weight: bold; }
        .priority-medium { color: #ffc107; }
        .priority-low { color: #28a745; }
        .status-pending { background-color: #6c757d; }
        .status-in_progress { background-color: #007bff; }
        .status-completed { background-color: #28a745; }
        .overdue { color: #dc3545; font-weight: bold; }
        .card-task { transition: transform 0.2s; }
        .card-task:hover { transform: translateY(-5px); box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
    </style>
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-dark bg-primary">
        <div class="container">
            <a class="navbar-brand" href="{% url \'tasks:task_list\' %}">Task Manager</a>
            <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
                <span class="navbar-toggler-icon"></span>
            </button>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav me-auto">
                    <li class="nav-item">
                        <a class="nav-link" href="{% url \'tasks:task_list\' %}">All Tasks</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="{% url \'tasks:task_create\' %}">New Task</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="{% url \'tasks:category_list\' %}">Categories</a>
                    </li>
                </ul>
            </div>
        </div>
    </nav>

    <main class="container mt-4">
        {% if messages %}
            {% for message in messages %}
                <div class="alert alert-{{ message.tags }} alert-dismissible fade show" role="alert">
                    {{ message }}
                    <button type="button" class="btn-close" data-bs-dismiss="alert"></button>
                </div>
            {% endfor %}
        {% endif %}
        
        {% block content %}{% endblock %}
    </main>

    <footer class="container mt-5 mb-3 text-center text-muted">
        <hr>
        <p>Task Manager - Django Capstone Project</p>
    </footer>

    <!-- Bootstrap JS -->
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
'''

print(BASE_TEMPLATE)

### Step 7.3: Task List Template

Create `tasks/templates/tasks/task_list.html`:

In [ ]:
# tasks/templates/tasks/task_list.html

TASK_LIST_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}All Tasks - Task Manager{% endblock %}

{% block content %}
<div class="row mb-4">
    <div class="col">
        <h1>Tasks</h1>
    </div>
    <div class="col-auto">
        <a href="{% url \'tasks:task_create\' %}" class="btn btn-primary">+ New Task</a>
    </div>
</div>

<!-- Filters -->
<div class="card mb-4">
    <div class="card-body">
        <form method="get" class="row g-3">
            <div class="col-md-3">
                <label class="form-label">Status</label>
                <select name="status" class="form-select">
                    <option value="">All Statuses</option>
                    {% for value, label in status_choices %}
                        <option value="{{ value }}" {% if current_status == value %}selected{% endif %}>
                            {{ label }}
                        </option>
                    {% endfor %}
                </select>
            </div>
            <div class="col-md-3">
                <label class="form-label">Priority</label>
                <select name="priority" class="form-select">
                    <option value="">All Priorities</option>
                    {% for value, label in priority_choices %}
                        <option value="{{ value }}" {% if current_priority == value %}selected{% endif %}>
                            {{ label }}
                        </option>
                    {% endfor %}
                </select>
            </div>
            <div class="col-md-3">
                <label class="form-label">Category</label>
                <select name="category" class="form-select">
                    <option value="">All Categories</option>
                    {% for cat in categories %}
                        <option value="{{ cat.id }}" {% if current_category == cat.id|stringformat:"s" %}selected{% endif %}>
                            {{ cat.name }}
                        </option>
                    {% endfor %}
                </select>
            </div>
            <div class="col-md-3 d-flex align-items-end">
                <button type="submit" class="btn btn-secondary me-2">Filter</button>
                <a href="{% url \'tasks:task_list\' %}" class="btn btn-outline-secondary">Clear</a>
            </div>
        </form>
    </div>
</div>

<!-- Task List -->
{% if tasks %}
    <div class="row">
        {% for task in tasks %}
            <div class="col-md-6 col-lg-4 mb-4">
                <div class="card card-task h-100">
                    <div class="card-header d-flex justify-content-between align-items-center">
                        <span class="badge status-{{ task.status }}">{{ task.get_status_display }}</span>
                        <span class="priority-{{ task.priority }}">{{ task.get_priority_display }}</span>
                    </div>
                    <div class="card-body">
                        <h5 class="card-title">
                            <a href="{% url \'tasks:task_detail\' task.pk %}" class="text-decoration-none">
                                {{ task.title }}
                            </a>
                        </h5>
                        {% if task.description %}
                            <p class="card-text text-muted">
                                {{ task.description|truncatewords:20 }}
                            </p>
                        {% endif %}
                        {% if task.category %}
                            <span class="badge bg-info">{{ task.category.name }}</span>
                        {% endif %}
                    </div>
                    <div class="card-footer text-muted">
                        {% if task.due_date %}
                            <small class="{% if task.is_overdue %}overdue{% endif %}">
                                Due: {{ task.due_date }}
                                {% if task.is_overdue %}(Overdue!){% endif %}
                            </small>
                        {% else %}
                            <small>No due date</small>
                        {% endif %}
                    </div>
                </div>
            </div>
        {% endfor %}
    </div>

    <!-- Pagination -->
    {% if page_obj.has_other_pages %}
        <nav aria-label="Task pagination">
            <ul class="pagination justify-content-center">
                {% if page_obj.has_previous %}
                    <li class="page-item">
                        <a class="page-link" href="?page={{ page_obj.previous_page_number }}">Previous</a>
                    </li>
                {% endif %}
                
                {% for num in page_obj.paginator.page_range %}
                    <li class="page-item {% if page_obj.number == num %}active{% endif %}">
                        <a class="page-link" href="?page={{ num }}">{{ num }}</a>
                    </li>
                {% endfor %}
                
                {% if page_obj.has_next %}
                    <li class="page-item">
                        <a class="page-link" href="?page={{ page_obj.next_page_number }}">Next</a>
                    </li>
                {% endif %}
            </ul>
        </nav>
    {% endif %}
{% else %}
    <div class="alert alert-info">
        <h4>No tasks found</h4>
        <p>Get started by <a href="{% url \'tasks:task_create\' %}">creating your first task</a>!</p>
    </div>
{% endif %}
{% endblock %}
'''

print(TASK_LIST_TEMPLATE)

### Step 7.4: Task Detail Template

Create `tasks/templates/tasks/task_detail.html`:

In [ ]:
# tasks/templates/tasks/task_detail.html

TASK_DETAIL_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}{{ task.title }} - Task Manager{% endblock %}

{% block content %}
<div class="row">
    <div class="col-lg-8">
        <nav aria-label="breadcrumb">
            <ol class="breadcrumb">
                <li class="breadcrumb-item"><a href="{% url \'tasks:task_list\' %}">Tasks</a></li>
                <li class="breadcrumb-item active">{{ task.title|truncatewords:5 }}</li>
            </ol>
        </nav>

        <div class="card">
            <div class="card-header d-flex justify-content-between align-items-center">
                <h2 class="mb-0">{{ task.title }}</h2>
                <div>
                    <a href="{% url \'tasks:task_update\' task.pk %}" class="btn btn-warning btn-sm">Edit</a>
                    <a href="{% url \'tasks:task_delete\' task.pk %}" class="btn btn-danger btn-sm">Delete</a>
                </div>
            </div>
            <div class="card-body">
                <div class="row mb-3">
                    <div class="col-md-4">
                        <strong>Status:</strong>
                        <span class="badge status-{{ task.status }}">{{ task.get_status_display }}</span>
                    </div>
                    <div class="col-md-4">
                        <strong>Priority:</strong>
                        <span class="priority-{{ task.priority }}">{{ task.get_priority_display }}</span>
                    </div>
                    <div class="col-md-4">
                        <strong>Category:</strong>
                        {% if task.category %}
                            <span class="badge bg-info">{{ task.category.name }}</span>
                        {% else %}
                            <span class="text-muted">None</span>
                        {% endif %}
                    </div>
                </div>

                <div class="row mb-3">
                    <div class="col-md-6">
                        <strong>Due Date:</strong>
                        {% if task.due_date %}
                            <span class="{% if task.is_overdue %}overdue{% endif %}">
                                {{ task.due_date }}
                                {% if task.is_overdue %}(Overdue!){% endif %}
                            </span>
                        {% else %}
                            <span class="text-muted">Not set</span>
                        {% endif %}
                    </div>
                    <div class="col-md-6">
                        <strong>Created:</strong>
                        {{ task.created_at|date:"N j, Y, P" }}
                    </div>
                </div>

                <hr>

                <h5>Description</h5>
                {% if task.description %}
                    <p>{{ task.description|linebreaks }}</p>
                {% else %}
                    <p class="text-muted">No description provided.</p>
                {% endif %}

                <hr>

                <small class="text-muted">
                    Last updated: {{ task.updated_at|date:"N j, Y, P" }}
                </small>
            </div>
        </div>

        <div class="mt-3">
            <a href="{% url \'tasks:task_list\' %}" class="btn btn-secondary">Back to Tasks</a>
        </div>
    </div>
</div>
{% endblock %}
'''

print(TASK_DETAIL_TEMPLATE)

### Step 7.5: Task Form Template

Create `tasks/templates/tasks/task_form.html`:

In [ ]:
# tasks/templates/tasks/task_form.html

TASK_FORM_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}{{ page_title }} - Task Manager{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <nav aria-label="breadcrumb">
            <ol class="breadcrumb">
                <li class="breadcrumb-item"><a href="{% url \'tasks:task_list\' %}">Tasks</a></li>
                <li class="breadcrumb-item active">{{ page_title }}</li>
            </ol>
        </nav>

        <div class="card">
            <div class="card-header">
                <h2 class="mb-0">{{ page_title }}</h2>
            </div>
            <div class="card-body">
                <form method="post" novalidate>
                    {% csrf_token %}
                    
                    {% for field in form %}
                        <div class="mb-3">
                            <label for="{{ field.id_for_label }}" class="form-label">
                                {{ field.label }}
                                {% if field.field.required %}<span class="text-danger">*</span>{% endif %}
                            </label>
                            {{ field }}
                            {% if field.help_text %}
                                <small class="form-text text-muted">{{ field.help_text }}</small>
                            {% endif %}
                            {% if field.errors %}
                                {% for error in field.errors %}
                                    <div class="invalid-feedback d-block">{{ error }}</div>
                                {% endfor %}
                            {% endif %}
                        </div>
                    {% endfor %}

                    <div class="d-flex gap-2">
                        <button type="submit" class="btn btn-primary">{{ button_text }}</button>
                        <a href="{% url \'tasks:task_list\' %}" class="btn btn-secondary">Cancel</a>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
'''

print(TASK_FORM_TEMPLATE)

### Step 7.6: Task Delete Confirmation Template

Create `tasks/templates/tasks/task_confirm_delete.html`:

In [ ]:
# tasks/templates/tasks/task_confirm_delete.html

TASK_DELETE_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}Delete Task - Task Manager{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card border-danger">
            <div class="card-header bg-danger text-white">
                <h4 class="mb-0">Confirm Deletion</h4>
            </div>
            <div class="card-body">
                <p class="lead">Are you sure you want to delete this task?</p>
                
                <div class="card mb-3">
                    <div class="card-body">
                        <h5>{{ task.title }}</h5>
                        <p class="text-muted mb-0">
                            {{ task.description|truncatewords:30|default:"No description" }}
                        </p>
                    </div>
                </div>
                
                <p class="text-danger"><strong>This action cannot be undone.</strong></p>
                
                <form method="post">
                    {% csrf_token %}
                    <div class="d-flex gap-2">
                        <button type="submit" class="btn btn-danger">Yes, Delete Task</button>
                        <a href="{% url \'tasks:task_detail\' task.pk %}" class="btn btn-secondary">Cancel</a>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
'''

print(TASK_DELETE_TEMPLATE)

### Step 7.7: Category Templates (Bonus)

Create `tasks/templates/tasks/category_list.html` and `category_form.html`:

In [ ]:
# tasks/templates/tasks/category_list.html

CATEGORY_LIST_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}Categories - Task Manager{% endblock %}

{% block content %}
<div class="row mb-4">
    <div class="col">
        <h1>Categories</h1>
    </div>
    <div class="col-auto">
        <a href="{% url \'tasks:category_create\' %}" class="btn btn-primary">+ New Category</a>
    </div>
</div>

{% if categories %}
    <div class="row">
        {% for category in categories %}
            <div class="col-md-4 mb-4">
                <div class="card">
                    <div class="card-body">
                        <h5 class="card-title">{{ category.name }}</h5>
                        <p class="card-text text-muted">
                            {{ category.description|default:"No description" }}
                        </p>
                        <p class="card-text">
                            <small>{{ category.tasks.count }} task(s)</small>
                        </p>
                        <a href="{% url \'tasks:task_list\' %}?category={{ category.id }}" 
                           class="btn btn-outline-primary btn-sm">View Tasks</a>
                    </div>
                </div>
            </div>
        {% endfor %}
    </div>
{% else %}
    <div class="alert alert-info">
        <p>No categories yet. <a href="{% url \'tasks:category_create\' %}">Create one!</a></p>
    </div>
{% endif %}
{% endblock %}
'''

print(CATEGORY_LIST_TEMPLATE)

In [ ]:
# tasks/templates/tasks/category_form.html

CATEGORY_FORM_TEMPLATE = '''
{% extends "tasks/base.html" %}

{% block title %}Create Category - Task Manager{% endblock %}

{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card">
            <div class="card-header">
                <h2 class="mb-0">Create New Category</h2>
            </div>
            <div class="card-body">
                <form method="post" novalidate>
                    {% csrf_token %}
                    
                    {% for field in form %}
                        <div class="mb-3">
                            <label for="{{ field.id_for_label }}" class="form-label">
                                {{ field.label }}
                            </label>
                            {{ field }}
                            {% if field.errors %}
                                {% for error in field.errors %}
                                    <div class="invalid-feedback d-block">{{ error }}</div>
                                {% endfor %}
                            {% endif %}
                        </div>
                    {% endfor %}

                    <div class="d-flex gap-2">
                        <button type="submit" class="btn btn-primary">Create Category</button>
                        <a href="{% url \'tasks:category_list\' %}" class="btn btn-secondary">Cancel</a>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
'''

print(CATEGORY_FORM_TEMPLATE)

---
## Section 8: Run and Test the Application

### Step 8.1: Start the Development Server

In [ ]:
# Terminal commands to run the application

# 1. Make sure migrations are applied
# python manage.py migrate

# 2. Create a superuser (if not already done)
# python manage.py createsuperuser

# 3. Start the development server
# python manage.py runserver

print("Run the server: python manage.py runserver")
print("")
print("Access the application at:")
print("  - Task List: http://127.0.0.1:8000/tasks/")
print("  - Admin: http://127.0.0.1:8000/admin/")

### Step 8.2: Testing CRUD Operations

Follow these steps to test all functionality:

1. **Create a Category**
   - Go to Categories page
   - Click "+ New Category"
   - Enter name: "Work" and description: "Work-related tasks"
   - Submit

2. **Create a Task**
   - Go to Tasks page
   - Click "+ New Task"
   - Fill in:
     - Title: "Complete Django project"
     - Description: "Finish the Task Manager capstone"
     - Status: In Progress
     - Priority: High
     - Due Date: (select a date)
     - Category: Work
   - Submit

3. **Read Task Details**
   - Click on the task title to view details
   - Verify all information is displayed correctly

4. **Update a Task**
   - Click "Edit" on the task detail page
   - Change status to "Completed"
   - Submit

5. **Delete a Task**
   - Click "Delete" on the task detail page
   - Confirm deletion

6. **Test Filters**
   - Create multiple tasks with different statuses/priorities
   - Use the filter dropdowns on the task list page

In [ ]:
# Optional: Django Shell Commands for Testing

# Access the Django shell:
# python manage.py shell

SHELL_COMMANDS = '''
# In the Django shell:

from tasks.models import Category, Task
from datetime import date, timedelta

# Create categories
work = Category.objects.create(name="Work", description="Work tasks")
personal = Category.objects.create(name="Personal", description="Personal tasks")

# Create tasks
Task.objects.create(
    title="Review project code",
    description="Code review for the new feature",
    status="pending",
    priority="high",
    due_date=date.today() + timedelta(days=2),
    category=work
)

Task.objects.create(
    title="Buy groceries",
    status="pending",
    priority="medium",
    category=personal
)

# Query tasks
Task.objects.filter(status="pending")
Task.objects.filter(priority="high")
work.tasks.all()  # Tasks in Work category
'''

print(SHELL_COMMANDS)

---
## Section 9: Writing Tests

Create tests for your models and views.

### Step 9.1: Model Tests

Create `tasks/tests.py`:

In [ ]:
# tasks/tests.py

TESTS_CODE = '''
from django.test import TestCase, Client
from django.urls import reverse
from datetime import date, timedelta
from .models import Category, Task


class CategoryModelTest(TestCase):
    """Tests for the Category model."""
    
    def setUp(self):
        """Set up test data."""
        self.category = Category.objects.create(
            name="Test Category",
            description="A test category"
        )
    
    def test_category_creation(self):
        """Test that a category can be created."""
        self.assertEqual(self.category.name, "Test Category")
        self.assertEqual(str(self.category), "Test Category")
    
    def test_category_name_unique(self):
        """Test that category names must be unique."""
        with self.assertRaises(Exception):
            Category.objects.create(name="Test Category")


class TaskModelTest(TestCase):
    """Tests for the Task model."""
    
    def setUp(self):
        """Set up test data."""
        self.category = Category.objects.create(name="Work")
        self.task = Task.objects.create(
            title="Test Task",
            description="A test task",
            status=Task.STATUS_PENDING,
            priority=Task.PRIORITY_HIGH,
            due_date=date.today() + timedelta(days=7),
            category=self.category
        )
    
    def test_task_creation(self):
        """Test that a task can be created."""
        self.assertEqual(self.task.title, "Test Task")
        self.assertEqual(str(self.task), "Test Task")
    
    def test_task_default_status(self):
        """Test that default status is pending."""
        task = Task.objects.create(title="New Task")
        self.assertEqual(task.status, Task.STATUS_PENDING)
    
    def test_task_is_overdue(self):
        """Test is_overdue property."""
        # Task with future due date
        self.assertFalse(self.task.is_overdue)
        
        # Task with past due date
        self.task.due_date = date.today() - timedelta(days=1)
        self.task.save()
        self.assertTrue(self.task.is_overdue)
        
        # Completed task is never overdue
        self.task.status = Task.STATUS_COMPLETED
        self.task.save()
        self.assertFalse(self.task.is_overdue)
    
    def test_task_get_absolute_url(self):
        """Test get_absolute_url method."""
        url = self.task.get_absolute_url()
        self.assertEqual(url, f"/tasks/task/{self.task.pk}/")


class TaskViewTest(TestCase):
    """Tests for Task views."""
    
    def setUp(self):
        """Set up test data and client."""
        self.client = Client()
        self.task = Task.objects.create(
            title="View Test Task",
            description="Testing views",
            status=Task.STATUS_PENDING,
            priority=Task.PRIORITY_MEDIUM
        )
    
    def test_task_list_view(self):
        """Test task list view."""
        response = self.client.get(reverse('tasks:task_list'))
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "View Test Task")
        self.assertTemplateUsed(response, 'tasks/task_list.html')
    
    def test_task_detail_view(self):
        """Test task detail view."""
        response = self.client.get(
            reverse('tasks:task_detail', kwargs={'pk': self.task.pk})
        )
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "View Test Task")
        self.assertTemplateUsed(response, 'tasks/task_detail.html')
    
    def test_task_create_view(self):
        """Test task create view."""
        response = self.client.get(reverse('tasks:task_create'))
        self.assertEqual(response.status_code, 200)
        
        # Test POST
        response = self.client.post(reverse('tasks:task_create'), {
            'title': 'New Created Task',
            'status': Task.STATUS_PENDING,
            'priority': Task.PRIORITY_LOW
        })
        self.assertEqual(response.status_code, 302)  # Redirect on success
        self.assertTrue(Task.objects.filter(title='New Created Task').exists())
    
    def test_task_update_view(self):
        """Test task update view."""
        response = self.client.post(
            reverse('tasks:task_update', kwargs={'pk': self.task.pk}),
            {
                'title': 'Updated Task Title',
                'status': Task.STATUS_COMPLETED,
                'priority': Task.PRIORITY_HIGH
            }
        )
        self.assertEqual(response.status_code, 302)
        self.task.refresh_from_db()
        self.assertEqual(self.task.title, 'Updated Task Title')
        self.assertEqual(self.task.status, Task.STATUS_COMPLETED)
    
    def test_task_delete_view(self):
        """Test task delete view."""
        task_pk = self.task.pk
        response = self.client.post(
            reverse('tasks:task_delete', kwargs={'pk': task_pk})
        )
        self.assertEqual(response.status_code, 302)
        self.assertFalse(Task.objects.filter(pk=task_pk).exists())
'''

print(TESTS_CODE)

In [ ]:
# Run tests with:
# python manage.py test tasks

# For verbose output:
# python manage.py test tasks -v 2

print("Run tests: python manage.py test tasks")
print("Run with coverage: coverage run manage.py test tasks && coverage report")

---
## Section 10: Challenges and Extensions

Extend your Task Manager with these optional features:

### Challenge 1: User Authentication
Add user accounts so each user has their own tasks.

**Steps:**
1. Add `user` ForeignKey to Task model
2. Use `LoginRequiredMixin` in views
3. Filter tasks by logged-in user
4. Create login/logout/register views

In [ ]:
# Challenge 1: User Authentication

AUTH_EXTENSION = '''
# models.py - Add user field
from django.contrib.auth.models import User

class Task(models.Model):
    # ... existing fields ...
    user = models.ForeignKey(
        User,
        on_delete=models.CASCADE,
        related_name='tasks'
    )

# views.py - Restrict to logged-in users
from django.contrib.auth.mixins import LoginRequiredMixin

class TaskListView(LoginRequiredMixin, ListView):
    model = Task
    
    def get_queryset(self):
        return Task.objects.filter(user=self.request.user)

class TaskCreateView(LoginRequiredMixin, CreateView):
    model = Task
    form_class = TaskForm
    
    def form_valid(self, form):
        form.instance.user = self.request.user
        return super().form_valid(form)
'''

print(AUTH_EXTENSION)

### Challenge 2: Search Functionality
Add a search box to find tasks by title or description.

In [ ]:
# Challenge 2: Search Functionality

SEARCH_EXTENSION = '''
# views.py - Add search to TaskListView
from django.db.models import Q

class TaskListView(ListView):
    model = Task
    
    def get_queryset(self):
        queryset = super().get_queryset()
        
        # Search
        search = self.request.GET.get('search')
        if search:
            queryset = queryset.filter(
                Q(title__icontains=search) |
                Q(description__icontains=search)
            )
        
        return queryset

# task_list.html - Add search form
# <form method="get" class="d-flex">
#     <input type="text" name="search" class="form-control" 
#            placeholder="Search tasks..." value="{{ request.GET.search }}">
#     <button type="submit" class="btn btn-primary">Search</button>
# </form>
'''

print(SEARCH_EXTENSION)

### Challenge 3: Task Sorting
Allow users to sort tasks by different fields.

In [ ]:
# Challenge 3: Task Sorting

SORTING_EXTENSION = '''
# views.py - Add sorting to TaskListView

class TaskListView(ListView):
    model = Task
    
    VALID_SORT_FIELDS = ['title', 'due_date', 'priority', 'status', 'created_at']
    
    def get_queryset(self):
        queryset = super().get_queryset()
        
        # Sorting
        sort_by = self.request.GET.get('sort', '-created_at')
        sort_field = sort_by.lstrip('-')
        
        if sort_field in self.VALID_SORT_FIELDS:
            queryset = queryset.order_by(sort_by)
        
        return queryset
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['current_sort'] = self.request.GET.get('sort', '-created_at')
        return context
'''

print(SORTING_EXTENSION)

### Challenge 4: Due Date Reminders
Highlight tasks that are due soon.

In [ ]:
# Challenge 4: Due Date Reminders

REMINDER_EXTENSION = '''
# models.py - Add properties

class Task(models.Model):
    # ... existing code ...
    
    @property
    def is_due_soon(self) -> bool:
        """Check if task is due within 3 days."""
        if self.due_date and self.status != self.STATUS_COMPLETED:
            days_until_due = (self.due_date - timezone.now().date()).days
            return 0 <= days_until_due <= 3
        return False
    
    @property
    def days_until_due(self) -> int | None:
        """Return number of days until due date."""
        if self.due_date:
            return (self.due_date - timezone.now().date()).days
        return None

# In template:
# {% if task.is_due_soon %}
#     <span class="badge bg-warning">Due in {{ task.days_until_due }} day(s)!</span>
# {% endif %}
'''

print(REMINDER_EXTENSION)

### Challenge 5: REST API
Add a REST API using Django REST Framework.

In [ ]:
# Challenge 5: REST API with Django REST Framework

API_EXTENSION = '''
# Install: pip install djangorestframework
# Add 'rest_framework' to INSTALLED_APPS

# tasks/serializers.py
from rest_framework import serializers
from .models import Task, Category

class CategorySerializer(serializers.ModelSerializer):
    class Meta:
        model = Category
        fields = ['id', 'name', 'description', 'created_at']

class TaskSerializer(serializers.ModelSerializer):
    category_name = serializers.CharField(source='category.name', read_only=True)
    
    class Meta:
        model = Task
        fields = [
            'id', 'title', 'description', 'status', 'priority',
            'due_date', 'category', 'category_name', 'created_at', 'updated_at'
        ]

# tasks/api_views.py
from rest_framework import viewsets
from .models import Task, Category
from .serializers import TaskSerializer, CategorySerializer

class TaskViewSet(viewsets.ModelViewSet):
    queryset = Task.objects.all()
    serializer_class = TaskSerializer

class CategoryViewSet(viewsets.ModelViewSet):
    queryset = Category.objects.all()
    serializer_class = CategorySerializer

# tasks/urls.py - Add API routes
from rest_framework.routers import DefaultRouter
from . import api_views

router = DefaultRouter()
router.register(r'tasks', api_views.TaskViewSet)
router.register(r'categories', api_views.CategoryViewSet)

urlpatterns = [
    # ... existing URLs ...
    path('api/', include(router.urls)),
]
'''

print(API_EXTENSION)

### More Challenge Ideas

6. **Task Tags**: Add a ManyToMany relationship for tags
7. **Task Comments**: Add a Comment model related to tasks
8. **File Attachments**: Allow attaching files to tasks
9. **Email Notifications**: Send emails when tasks are due
10. **Task Templates**: Save reusable task templates
11. **Dashboard**: Create a dashboard with task statistics
12. **Export**: Export tasks to CSV or PDF
13. **Drag-and-Drop**: Add drag-and-drop to change task status
14. **Dark Mode**: Add a dark mode toggle
15. **Subtasks**: Add subtasks as a self-referential ForeignKey

---
## Summary

### What We Built
A complete Task Manager web application with:
- **Models**: Category and Task with relationships
- **Admin**: Full admin interface for data management
- **Views**: Class-based views for all CRUD operations
- **Templates**: Bootstrap-styled responsive templates
- **Forms**: ModelForms with validation
- **URLs**: Named URL patterns with namespacing
- **Tests**: Comprehensive test suite

### Key Django Concepts Demonstrated
1. **MTV Pattern**: Models, Templates, Views
2. **ORM**: Model definitions, relationships, queries
3. **Migrations**: Database schema management
4. **CBVs**: ListView, DetailView, CreateView, UpdateView, DeleteView
5. **Template Inheritance**: Base templates with blocks
6. **URL Routing**: Named URLs with namespaces
7. **Forms**: ModelForms, validation, widgets
8. **Admin**: Model registration and customization
9. **Messages**: Flash messages for user feedback

### Next Steps
1. Deploy to a production server (Heroku, Railway, PythonAnywhere)
2. Add user authentication
3. Implement the challenge extensions
4. Add a REST API for mobile apps
5. Write more comprehensive tests

### Resources
- [Django Documentation](https://docs.djangoproject.com/)
- [Django REST Framework](https://www.django-rest-framework.org/)
- [Bootstrap Documentation](https://getbootstrap.com/docs/)
- [MDN Web Docs - Django Tutorial](https://developer.mozilla.org/en-US/docs/Learn/Server-side/Django)